# Desafio Mercado libre

En este notebook se recolectan los datos usando la api de search de mercado libre.


* Los calculos se realizaran sobre publicaciones de Argentina usando moneda Argentina.

* Se descargaron 1000 productos por categoria
    
    
Output del notebook: df_total_original




In [1]:
import numpy as np
import pandas as pd
import time
from time import process_time

In [2]:
from functions import request_a_diccionario, generar_df

Para descargar los datos vamos a obtener 1000 items por categoria y guardarlos en un df.
Usamos la API (itemps y busquedas):

/sites/ SITE_ID/search?category=$CATEGORY_ID

Los valores por defecto son Offset=0 y Limit=50.
Vamos a dejar el limite de 50 y cambiando el offset para mostrar 20 paginas

Obtener ítems listados en una categoría.

In [3]:
lista_categorias = request_a_diccionario("https://api.mercadolibre.com/sites/MLA/categories")

In [4]:
paginas_cant=20
limite=50
t_inicial=time.perf_counter()
lista_dfs=[]
for numero_categoria,categoria in enumerate(lista_categorias):
    print("Recolectando data de la categoría:", str(numero_categoria), "de", len(lista_categorias), "(categoría", categoria["name"], ").")
    for pagina in range(paginas_cant):
        print("Página:" + str(pagina) + " de 19.", end='\r', flush=True)
        categoria_id=categoria["id"]
        categoria_name=categoria["name"]
        offset=str(limite*pagina+1)
        items=request_a_diccionario("https://api.mercadolibre.com/sites/MLA/search?category=" + categoria_id + "&limit=50&offset=" + offset)
        df=generar_df(items)
        df["categoria_principal"]=categoria_name
        lista_dfs.append(df)
elapsed_time=time.perf_counter()-t_inicial
print("tiempo total de ejecucion:",round(elapsed_time/60,1)," minutos.")



Recolectando data de la categoría: 0 de 32 (categoría Accesorios para Vehículos ).
Recolectando data de la categoría: 1 de 32 (categoría Agro ).
Recolectando data de la categoría: 2 de 32 (categoría Alimentos y Bebidas ).
Recolectando data de la categoría: 3 de 32 (categoría Animales y Mascotas ).
Recolectando data de la categoría: 4 de 32 (categoría Antigüedades y Colecciones ).
Recolectando data de la categoría: 5 de 32 (categoría Arte, Librería y Mercería ).
Recolectando data de la categoría: 6 de 32 (categoría Autos, Motos y Otros ).
Recolectando data de la categoría: 7 de 32 (categoría Bebés ).
Recolectando data de la categoría: 8 de 32 (categoría Belleza y Cuidado Personal ).
Recolectando data de la categoría: 9 de 32 (categoría Cámaras y Accesorios ).
Recolectando data de la categoría: 10 de 32 (categoría Celulares y Teléfonos ).
Recolectando data de la categoría: 11 de 32 (categoría Computación ).
Recolectando data de la categoría: 12 de 32 (categoría Consolas y Videojuegos ).


hasta ahora hay una lista de dataframes cada elemento un df con los items de una pagina (en las 20 primeras paginas)
vamos a concatenar todo en un solo df

In [5]:
df_total_original=pd.concat(lista_dfs,axis=0)

In [6]:
len(df_total_original)

31970

In [7]:
len(lista_categorias)

32

El tamaño del df esta ok. 

El limite= 50 resultados por pagina en 20 paginas = cantidad de resultados por categorias

In [8]:
df_total_original.head()

,id,site_id,title,price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,...,address__city_name,shipping__free_shipping,shipping__store_pick_up,original_price,category_id,official_store_id,catalog_product_id,attributes__marca,attributes__modelo,categoria_principal
0,MLA610309020,MLA,Sensor De Estacionamiento * Colores Con Displa...,1729.80,ARS,50,5000,buy_it_now,gold_special,new,...,Liniers,False,False,NaN,MLA45905,NaN,None,Lefer,XD-068,Accesorios para Vehículos
1,MLA1103800748,MLA,Casco Para Moto Integral Halcon H57 Blanco Ta...,3499.00,ARS,1,47,buy_it_now,gold_special,new,...,Floresta,False,True,NaN,MLA4712,NaN,MLA11386905,Halcon,H57,Accesorios para Vehículos
2,MLA636709517,MLA,Cubre Alfombra Goma Pesada Universal 3 Piezas ...,2159.99,ARS,50,5000,buy_it_now,gold_special,new,...,Ciudadela,False,False,NaN,MLA2219,NaN,None,At,Cubre alfombra,Accesorios para Vehículos
3,MLA634474528,MLA,Aceite Motul 8100 X-cess 5w40 X 5 Lts.,7810.00,ARS,1,5000,buy_it_now,gold_special,new,...,Caseros,True,False,NaN,MLA375007,NaN,MLA10427504,Motul,Auto,Accesorios para Vehículos
4,MLA909004508,MLA,"Estéreo Para Auto Electroland Orgt Con Usb, Bl...",3470.11,ARS,79,6020,buy_it_now,gold_special,new,...,Boedo,False,False,3899.0,MLA6867,NaN,MLA16999186,None,None,Accesorios para Vehículos


In [9]:
#ver cuantos valores duplicados y eliminar 
df_total_original.duplicated(keep="first").sum()

179

In [10]:
df_total_original=df_total_original[~df_total_original.duplicated(keep="first")]

In [11]:
len(df_total_original)

31791

In [12]:
df_total_original.to_csv("df_total_original.csv",index="False")

finalmente guardamos csv para realizar el analisis